Table 4.12.1: Parental status of new presentations to treatment 2017-18	

In [ ]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [ ]:
tab = tabs['Table 4.12.1']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [ ]:
observations = tab.excel_ref('B14').expand(DOWN).expand(RIGHT).is_not_blank() - tab.excel_ref('B22').expand(DOWN).expand(RIGHT)
observations

In [ ]:
PS = tab.excel_ref('A14').expand(DOWN).is_not_blank() - tab.excel_ref('A20').expand(DOWN)
PS

In [ ]:
Clients = tab.excel_ref('B12').expand(RIGHT).is_not_blank()
Clients

In [ ]:
Sex = tab.excel_ref('B13').expand(RIGHT).is_not_blank()
Sex

In [ ]:
Dimensions = [
            HDim(PS,'Parental status',DIRECTLY,LEFT),
            HDim(Clients,'Clients in new treatment',CLOSEST,LEFT),
            HDim(Sex,'Sex',CLOSEST,LEFT),
            HDimConst('Unit','People')            
            ]

In [ ]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# if is_interactive():
#     savepreviewhtml(c1)

In [ ]:
new_table = c1.topandas()
new_table

In [ ]:
new_table = new_table[new_table['OBS'] != 0 ]

In [ ]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [ ]:
new_table.head()

In [ ]:
new_table['Value'] = new_table['Value'].astype(str)

In [ ]:
new_table['Measure Type'] = 'Count'

In [ ]:
new_table.tail()

In [ ]:
new_table.dtypes

In [ ]:
new_table.head(3)

In [ ]:
new_table['Clients in new treatment'] = new_table['Clients in new treatment'].map(
    lambda x: {
        'Total' : 'All Clients' 
        }.get(x, x))

In [ ]:
def user_perc(x):
    
    if x == None:
        return 'All inclusice incomplete data'
    else:
        return x
    
new_table['Parental status'] = new_table.apply(lambda row: user_perc(row['Parental status']), axis = 1)

In [ ]:
new_table.head(3)

In [ ]:
new_table['Parental status'] = new_table['Parental status'].map(
    lambda x: {
        'Total' : 'All children',
        }.get(x, x))

In [ ]:
new_table['Parental status'] =  'Parental status/' + new_table['Parental status']
new_table['Clients in new treatment'] = 'New clients/' + new_table['Clients in new treatment']

In [ ]:
new_table['Parental status'] =  new_table['Parental status'] +'/'+ new_table['Sex']

In [ ]:
new_table.columns = ['Clients in treatment' if x=='Clients in new treatment' else x for x in new_table.columns]
new_table['Period'] = '2017-18'
new_table.columns = ['Basis of treatment' if x=='Parental status' else x for x in new_table.columns]
new_table['Substance'] = 'All'
new_table = new_table[['Period','Basis of treatment','Substance','Clients in treatment','Measure Type','Value','Unit']]

In [ ]:
new_table.head()